# Remove borders center crop and save numpy

Images have a lot of true black close to their margins, this step crops it to enhance the intensity normalization in step 4. We do not center crop adjusting to the prostate because our segmentation model works better with full pelvis images

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output
import itk
import SimpleITK as sitk
import matplotlib.patches as patches
from ipywidgets import interact, fixed

### View uncropped images

In [ ]:
def display_images(image1_z,image2_z, image1_npa, image2_npa, show_roi=False):
    # visualize cropped area with these parameters
    crop_size_x = 280 
    crop_size_y = 280
    offset_origin_x = 0
    offset_origin_y = 0
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    ax1 = plt.subplot(1,2,1)
    plt.imshow(image1_npa[image1_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('Image 1')
    plt.axis('off')

    if show_roi:
        # Add a square to represent the cropping area for the first image
        center1 = (image1_npa.shape[2] / 2, image1_npa.shape[1] / 2)
        rect1 = patches.Rectangle((center1[0] - crop_size_x / 2 + offset_origin_x, center1[1] - crop_size_y / 2 + offset_origin_y), 
                                crop_size_x, crop_size_y, linewidth=1, edgecolor='r', facecolor='none')
        ax1.add_patch(rect1)
    
    # Draw the moving image in the second subplot.
    ax2 = plt.subplot(1,2,2)
    plt.imshow(image2_npa[image2_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('Image 2')
    plt.axis('off')

    if show_roi:
        # Add a square to represent the cropping area for the second image
        center2 = (image2_npa.shape[2] / 2, image2_npa.shape[1] / 2)
        rect2 = patches.Rectangle((center2[0] - crop_size_x / 2 + offset_origin_x, center2[1] - crop_size_y / 2 + offset_origin_y), 
                                crop_size_x, crop_size_y, linewidth=1, edgecolor='r', facecolor='none')
        ax2.add_patch(rect2)
    
    plt.show()

In [ ]:
image1 = sitk.ReadImage("/local_ssd/practical_wise24/prostate_cancer/mha_registered_images/BREST_131/Pelvis_t2_haste_fs_db_tra_p2_320.mha", sitk.sitkFloat32) 
image2 = sitk.ReadImage("/local_ssd/practical_wise24/prostate_cancer/mha_registered_images/BREST_131/Pelvis_t2_spc_rst_tra_p2_iso.mha", sitk.sitkFloat32) 

In [ ]:

interact(display_images, image1_z=(0,image1.GetSize()[2]-1), image2_z=(0,image2.GetSize()[2]-1), image1_npa = fixed(sitk.GetArrayViewFromImage(image1)), image2_npa=fixed(sitk.GetArrayViewFromImage(image2), show_roi=False));

## Crop image

In [ ]:
def crop_sitk_image_centered(sitk_image, offset, size):
    # Get the size of the image
    image_size = sitk_image.GetSize()

    # Assume 3D image; for 2D, ignore the z-dimension
    image_center = [image_size[i] // 2 for i in range(len(image_size))]

    # Adjust the center by the offset
    adjusted_center = [image_center[i] + offset[i] for i in range(len(offset))]

    # Calculate the top-left corner of the crop region
    start_point = [adjusted_center[i] - size[i] // 2 for i in range(len(size))]

     # Using RegionOfInterest to crop the image
    roi_filter = sitk.RegionOfInterestImageFilter()
    roi_filter.SetSize(size)
    roi_filter.SetIndex(start_point)
    cropped_sitk_image = roi_filter.Execute(sitk_image)
    return cropped_sitk_image

### Crop all images

In [ ]:

centered_dataset_path = '/local_ssd/practical_wise24/prostate_cancer/clean_dataset'
# center_crop_dataset_path = './clean_dataset'
root_dir = '/local_ssd/practical_wise24/prostate_cancer/mha_registered_images/' # should be this one change
offset = (0, 0, 0)
crop_size = (280, 280, 72)
for patient_folder in os.listdir(root_dir):
    files_path = os.path.join(root_dir, patient_folder)
    write_file_path = os.path.join(center_crop_dataset_path, patient_folder)
    for mha_file_name in os.listdir(files_path):
        image = sitk.ReadImage(os.path.join(files_path, mha_file_name), sitk.sitkFloat32)
        cropped_image = crop_sitk_image_centered(image, offset, crop_size)
        if not os.path.exists(write_file_path):
            os.makedirs(write_file_path)
        sitk.WriteImage(cropped_image, os.path.join(write_file_path, mha_file_name))